In [388]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [389]:
!gdown --id '1MOC3LBT2Wo8uiFQEKd-y4IQImImoiWg6' --output train.csv
!gdown --id '1aOqUEZceg2XQmC58Ye-wLR_fdtzV_PGt' --output test.csv
!gdown --id '1Vt5g2oaeyK6BOjDEh3r8kVhxYjnJphqs' --output Gender.csv


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1MOC3LBT2Wo8uiFQEKd-y4IQImImoiWg6
To: /content/train.csv
100% 61.2k/61.2k [00:00<00:00, 43.5MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1aOqUEZceg2XQmC58Ye-wLR_fdtzV_PGt
To: /content/test.csv
100% 28.6k/28.6k [00:00<00:00, 33.8MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://

In [426]:
import pandas as pd
import numpy as np #np 轉 temsor較方便
import torch 
from pandas import DataFrame
from torch.utils.data import Dataset, DataLoader
from torch import nn
import random
from tqdm.auto import tqdm #進度條
from collections import Counter 
from torch.cuda.amp import autocast as autocast
from torch.nn.functional import normalize #均衡化
from keras import callbacks
from keras.callbacks import EarlyStopping 
from sklearn.preprocessing import MinMaxScaler

In [391]:
train_data = pd.read_csv('train.csv')
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [392]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [393]:
NAN = train_data.mode().iloc[0]

In [394]:
train_data.fillna(NAN,inplace=True)

In [395]:
train_data.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [396]:
train_data.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,28.566970,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.199572,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,24.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [397]:
# train_data.corr("pearson")

In [398]:
train_data = pd.get_dummies(train_data,columns=["Sex","Pclass","Embarked"])

In [434]:
Survived_tensor = torch.tensor(train_data.Survived)
S_feature = [col for col in train_data.columns]
S_feature.remove('PassengerId')
S_feature.remove('Survived')
S_feature.remove('Name')
S_feature.remove('Ticket')
S_feature.remove('Cabin')
S_feature.remove('Age')
S_feature.remove('SibSp')
S_feature.remove('Pclass_3')
S_feature.remove('Embarked_S')
S_feature
# train_data.head()


['Parch',
 'Fare',
 'Sex_female',
 'Sex_male',
 'Pclass_1',
 'Pclass_2',
 'Embarked_C',
 'Embarked_Q']

In [400]:
scaler = MinMaxScaler()
train_data[S_feature] = scaler.fit_transform(train_data[S_feature])

F_tensor = torch.tensor(train_data[S_feature].values,dtype = torch.float)
F_tensor
# F_N = normalize(F_tensor, p=2.0,dim=0)
# F_N

tensor([[0.0000, 0.0142, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1391, 1.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.0000, 0.0155, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.3333, 0.0458, 1.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0586, 0.0000,  ..., 0.0000, 1.0000, 0.0000],
        [0.0000, 0.0151, 0.0000,  ..., 0.0000, 0.0000, 1.0000]])

In [401]:
train_y = train_data.Survived
y_tensor =  torch.LongTensor(train_y.values)

In [402]:
#Use House_price code : https://www.kaggle.com/code/honglouis/lab-hw-housepirce/edit/run/103035425
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(8, 512)
        nn.Dropout(0,3)
        self.linear3 = nn.Linear(256, 128)
        nn.Dropout(0,3)
        self.linear4 = nn.Linear(128, 2)
        
    def forward(self, x):
        y_pred = self.linear1(x)
        y_pred = self.linear2(y_pred)
        y_pred = self.linear3(y_pred)
        y_pred = self.linear4(y_pred)
        return y_pred
model = Net()

In [403]:
criterion = nn.CrossEntropyLoss()

In [404]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [405]:
import random
def same_seeds(seed):
    # Python built-in random module
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    # Cuda
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

In [406]:
same_seeds = 63
losses = []
pre_loss=0
stop=0
num=20
for t in range(500):
    y_pred = model(F_tensor)
    loss = criterion(y_pred,y_tensor)
    print(t, "Loss =",loss.item())
    losses.append(loss.item())
    if torch.isnan(loss): #if inputdata(tensor) have NAN return
        break
    if loss - pre_loss < 0.001:
      stop+=1
      if stop >= num:
        print("Early Stopping")
        break
    else:
      stop=0
    pre_loss = loss
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

0 Loss = 0.7279185652732849
1 Loss = 1.5708891153335571
2 Loss = 10.156622886657715
3 Loss = 1.6661020517349243
4 Loss = 1.3577519655227661
5 Loss = 1.0285520553588867
6 Loss = 0.9787167906761169
7 Loss = 0.7540057301521301
8 Loss = 0.8017845749855042
9 Loss = 0.9533383250236511
10 Loss = 0.8288201093673706
11 Loss = 0.694310188293457
12 Loss = 0.8647165298461914
13 Loss = 0.6279988884925842
14 Loss = 0.49279487133026123
15 Loss = 0.6002482771873474
16 Loss = 0.6389312744140625
17 Loss = 0.5999947786331177
18 Loss = 0.5405334830284119
19 Loss = 0.49599963426589966
20 Loss = 0.49407926201820374
21 Loss = 0.5020927786827087
22 Loss = 0.4991733431816101
23 Loss = 0.4964250922203064
24 Loss = 0.49242931604385376
25 Loss = 0.48896920680999756
26 Loss = 0.47780317068099976
27 Loss = 0.46599313616752625
28 Loss = 0.46977099776268005
29 Loss = 0.47560915350914
30 Loss = 0.4773501753807068
31 Loss = 0.47355344891548157
32 Loss = 0.4690379798412323
33 Loss = 0.4745377004146576
34 Loss = 0.472678

In [452]:
train_test = pd.read_csv('test.csv')

train_test = pd.get_dummies(train_test,columns=["Sex","Pclass","Embarked"])
test_f = [col for col in train_test.columns]
test_f.remove('PassengerId')
test_f.remove('Name')
test_f.remove('Ticket')
test_f.remove('Cabin')
test_f.remove('Age')
test_f.remove('SibSp')
test_f.remove('Pclass_3')
test_f.remove('Embarked_S') #移除 -corr
# 處裡空值
NAN_test = train_test.mode().iloc[0] 
train_test.fillna(NAN_test,inplace=True)
train_test.isna().sum()
train_test.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,892,"Kelly, Mr. James",34.5,0,0,330911,7.8292,B57 B59 B63 B66,0,1,0,0,1,0,1,0
1,893,"Wilkes, Mrs. James (Ellen Needs)",47.0,1,0,363272,7.0000,B57 B59 B63 B66,1,0,0,0,1,0,0,1
2,894,"Myles, Mr. Thomas Francis",62.0,0,0,240276,9.6875,B57 B59 B63 B66,0,1,0,1,0,0,1,0
3,895,"Wirz, Mr. Albert",27.0,0,0,315154,8.6625,B57 B59 B63 B66,0,1,0,0,1,0,0,1
4,896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",22.0,1,1,3101298,12.2875,B57 B59 B63 B66,1,0,0,0,1,0,0,1


In [453]:
scaler = MinMaxScaler() 
train_test[test_f] = scaler.fit_transform(train_test[test_f]) #資料均衡化
test_tensor = torch.tensor(train_test[test_f].values,dtype = torch.float)

In [469]:
result = model(test_tensor)
result

tensor([[ 1.3767e+00, -6.3300e-01],
        [ 2.1895e+00,  2.5225e+00],
        [ 7.8526e-01, -3.2336e-02],
        [ 1.1235e+00, -1.3013e+00],
        [ 2.3031e+00,  2.5497e+00],
        [ 1.1242e+00, -1.2993e+00],
        [ 2.4447e+00,  3.1958e+00],
        [ 6.6356e-01, -6.2762e-01],
        [ 2.2889e+00,  3.1973e+00],
        [ 1.1445e+00, -1.2476e+00],
        [ 1.1224e+00, -1.3039e+00],
        [ 5.1222e-01, -6.2601e-02],
        [ 1.6568e+00,  3.9619e+00],
        [ 5.5297e-01, -6.4695e-01],
        [ 1.6282e+00,  3.8888e+00],
        [ 1.7227e+00,  3.8625e+00],
        [ 7.8887e-01, -2.3109e-02],
        [ 1.2206e+00, -6.3222e-01],
        [ 2.1907e+00,  2.5257e+00],
        [ 2.2889e+00,  3.1973e+00],
        [ 6.5663e-01,  7.2718e-01],
        [ 1.2225e+00, -1.3114e+00],
        [ 1.5882e+00,  3.7866e+00],
        [ 7.6583e-01,  7.4298e-01],
        [ 2.3197e+00,  5.2870e+00],
        [ 1.1314e+00, -1.2810e+00],
        [ 1.8349e+00,  4.5745e+00],
        [ 1.2206e+00, -6.322

In [ ]:
values, labels = torch.max(result, 1)
survived = labels.data.numpy()
submission = pd.DataFrame({'PassengerId': train_test['PassengerId'], 'Survived': survived})

In [467]:
submission.to_csv('/content/submission.csv', index=False)

# 新增區段